# EDA

## Preparing Data
- load multiple json files into dataframes
    - generate one dataframe with all the channels content

## Assess and Clean Data
- drop columns which doesn't make sense
- drop rows which doesn't make sense
- filter down for 1to1 convos


In [93]:
# import basic libraries
import os, json
import pandas as pd
import numpy as np
import glob
pd.options.mode.chained_assignment = None 
import networkx as nx
import matplotlib.pyplot as plt

### Loading Multiple Json Files into DataFrames
Generate one dataframe with all the channels in it

In [94]:
main_folder_path = '../raw_data' 

def parse_all_json(main_folder_path):
    df = pd.DataFrame()

### Iterate through the group of folders
    for folder in os.listdir(main_folder_path):
        folder_path = os.path.join(main_folder_path, folder)
        
        if os.path.isdir(folder_path):
## Iterate through each individual folder
            for file in os.listdir(folder_path):
                file = os.path.join(main_folder_path, folder, file)
            
            #add a channel name column and add which folder is the value coming from: general, labhelp, 
                if file.endswith('.json'):
                    data = pd.read_json(file)
                    data['channel_name'] = folder
                    df = pd.concat([df, data])
    return df

df1 = parse_all_json('../raw_data/')

In [95]:
# save to csv
df1.to_csv(r'../raw_data/data_raw.csv', index = False)

### Assessment

In [96]:
df1.shape

(5185, 38)

In [97]:
df1.head()

,type,subtype,ts,user,text,channel_name,client_msg_id,team,user_team,source_team,...,bot_id,bot_profile,inviter,purpose,topic,root,old_name,name,x_files,hidden
0,message,channel_join,1.616237e+09,U01S1CWGTU4,<@U01S1CWGTU4> has joined the channel,music,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,message,NaN,1.617897e+09,U01S7BM4N81,<https://www.youtube.com/watch?v=qW1eTP9CKSE>,music,98d2a1df-fcab-4a15-a96c-514cd4fa55fa,T01RBRV5F7H,T01RBRV5F7H,T01RBRV5F7H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,message,NaN,1.620900e+09,U01RXCQHMHT,A thematic name for a playlist\n\n<https://sou...,music,c097123a-a43a-47a7-8d6b-c4ed6574b2c6,T01RBRV5F7H,T01RBRV5F7H,T01RBRV5F7H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,message,NaN,1.620200e+09,U01S7KCL3DF,<@U01RXCQHMHT> here’s a bangin track about Nig...,music,a153f00e-8a2f-4c0f-9f4f-ebcf678f250e,T01RBRV5F7H,T01RBRV5F7H,T01RBRV5F7H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,message,NaN,1.620211e+09,U01RSRE0N3D,For all of you LotR ravers\n<https://www.youtu...,music,41475476-99d9-41d5-beb7-727785d8cd49,T01RBRV5F7H,T01RBRV5F7H,T01RBRV5F7H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
def real_name(x):
    """this function is applied to column user_profile to extract real_name
    """
    if x != x:
        return 'noname'
    else:
        return x['real_name']

df1['real_name'] = df1['user_profile'].apply(real_name)

# drop user_profile column
df1.drop('user_profile', axis=1, inplace=True) 

In [99]:
def cleaning(df1):
    """this function is cleans the dataframe from noise in the data
    """
    
    # filter out for the rows which has subtype values
    df1 = df1[(df1.subtype != 'channel_join') & 
                                (df1.subtype != 'channel_join') &
                                (df1.subtype != 'channel_purpose') &
                                (df1.subtype != 'thread_broadcast')]
    # drop subtype column with the values we don't need anymore
    df1.drop('subtype', axis=1, inplace=True) 
    
    # droppig unneccessary columns
    df1.drop(['type','client_msg_id', 'team', 'user_team', 
                'source_team','attachments', 'blocks', 'edited', 'reactions', 
                'is_locked', 'subscribed', 'last_read', 'parent_user_id',
                'files', 'upload', 'display_as_bot', 'bot_id', 'bot_profile', 'inviter',
                'purpose', 'topic', 'root', 'old_name', 'name', 'x_files', 'hidden'], axis=1, inplace=True)
    
    # reordering columns
    df1 = df1[['ts', 'user', 'real_name','text', 'channel_name',
               'thread_ts','reply_count', 'reply_users_count', 
               'latest_reply', 'reply_users','replies']]
    
    # filter out names that don't have a real name
    df1 = df1[(df1.real_name != 'noname')]
    
    return df1

In [100]:
df1 = cleaning(df1)
df = df1.copy()
df = df.dropna()

## Process the DF

### Separate direct conversations (where there is only 1 user who replied)

In [101]:
# see count of users replied to the messages in 'reply_len' column
df['reply_len'] = df['reply_users'].apply(lambda a : len(a))
df.shape

(426, 12)

There are 426 occasions when there is a thread.

In [102]:
# filter down where there is only 1 reply
df = df.loc[df['reply_len'] == 1][['text','ts', 'user', 'real_name','reply_users']].reset_index(drop=True)

# strip down brackets
df['reply_users'] = df['reply_users'].str[0]

# rename columns
df.rename(columns={'reply_users' : 'to'}, inplace=True)

# test
df.head()
df.to_csv('one_replies.csv')

In [103]:
df.shape

(86, 5)

There are 86 occasions when there is only 1 user (= one to one) having a conversation with another one.

## Out of scope below: 
- to separate messages by questions and answers

### Define function to filter questions

In [104]:
def is_question(txt):
    words = {'how', 'what', 'where', '?', 'when', 'anyone', 'which', 'can', 'do you', 'does'}
    txt = set(txt.lower().split())
    return len(words.intersection(txt)) >= 1

df['question'] = df['text'].apply(is_question)

df['question'].value_counts()

False    58
True     28
Name: question, dtype: int64

There are 28 questions from direct conversations.

### Define functions to filter answers


In [105]:
def is_answer(txt):
    gratitude_words = {'thanks' ,'thx' ,' thanx', 'thank you', 'thnx','thanks' ,'thx' , 'brilliant','thanks a lot', 
        'thank you very much', 'great' , 'awesome','cool'}
    txt = set(txt.lower().split())
    return len(gratitude_words.intersection(txt)) >= 1

df['answer'] = df['text'].apply(is_answer)

df['answer'].value_counts()

False    83
True      3
Name: answer, dtype: int64

There are 3 answers from direct conversations.